In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score,max_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures
from gplearn.genetic import SymbolicRegressor, SymbolicTransformer
import mlxtend
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import validation_curve
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
crossvalidation = KFold(n_splits=10, random_state=None, shuffle=False)
def calculate_adj_r2(r_sq, n, k):
    adj_r = 1-((1-r_sq)*(n-1)/(n-k-1))
    return adj_r

In [56]:
dataset = pd.read_csv("USA_Housing.csv")

In [57]:
dataset.shape

(5000, 6)

In [58]:
dataset.head()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price
0,79545.45857,5.682861,7.009188,4.09,23086.80050,1.059034e+06
1,79248.64245,6.002900,6.730821,3.09,40173.07217,1.505891e+06
2,61287.06718,5.865890,8.512727,5.13,36882.15940,1.058988e+06
3,63345.24005,7.188236,5.586729,3.26,34310.24283,1.260617e+06
4,59982.19723,5.040555,7.839388,4.23,26354.10947,6.309435e+05


In [59]:
dataset.isnull().sum()

Avg. Area Income                0
Avg. Area House Age             0
Avg. Area Number of Rooms       0
Avg. Area Number of Bedrooms    0
Area Population                 0
Price                           0
dtype: int64

In [60]:
dataset.describe()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5.000000e+03
mean,68583.108984,5.977222,6.987792,3.981330,36163.516039,1.232073e+06
std,10657.991214,0.991456,1.005833,1.234137,9925.650114,3.531176e+05
min,17796.631190,2.644304,3.236194,2.000000,172.610686,1.593866e+04
25%,61480.562390,5.322283,6.299250,3.140000,29403.928700,9.975771e+05
50%,68804.286405,5.970429,7.002902,4.050000,36199.406690,1.232669e+06
75%,75783.338665,6.650808,7.665871,4.490000,42861.290770,1.471210e+06
max,107701.748400,9.519088,10.759588,6.500000,69621.713380,2.469066e+06


In [61]:
# creating the training data

X = dataset.drop(['Price'], axis=1)
y = dataset['Price']

In [62]:
X.shape, y.shape

((5000, 5), (5000,))

<h2> Forward Feature Selection</h2>

In [63]:
sfs = SFS(LinearRegression(),
          k_features='best',
          forward=True,
          floating=False,
          verbose=2,
          scoring = 'r2',
          cv = 5)

In [64]:
sfs.fit(X, y)
print("Selected Features:",sfs.k_feature_names_)
print("Selected Features ID:",sfs.k_feature_idx_)

Selected Features: ('Avg. Area Income', 'Avg. Area House Age', 'Avg. Area Number of Rooms', 'Area Population')
Selected Features ID: (0, 1, 2, 4)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished

[2022-02-21 23:18:42] Features: 1/5 -- score: 0.4065836013812946[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished

[2022-02-21 23:18:42] Features: 2/5 -- score: 0.6124249771420225[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished

[2022-02-21 23:18:42] Features: 3/5 -- score: 0.7970040383531473[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 

In [65]:
df=pd.DataFrame.from_dict(sfs.get_metric_dict()).T
df

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
1,"(0,)","[0.4260936784765087, 0.4410126389901101, 0.364...",0.406584,"(Avg. Area Income,)",0.036804,0.028635,0.014317
2,"(0, 1)","[0.6264837396794845, 0.6566958614467223, 0.568...",0.612425,"(Avg. Area Income, Avg. Area House Age)",0.039491,0.030726,0.015363
3,"(0, 1, 4)","[0.799500115176274, 0.8146479457241405, 0.7881...",0.797004,"(Avg. Area Income, Avg. Area House Age, Area P...",0.012392,0.009641,0.004821
4,"(0, 1, 2, 4)","[0.917841711309842, 0.9202527128422192, 0.9153...",0.917583,"(Avg. Area Income, Avg. Area House Age, Avg. A...",0.003516,0.002736,0.001368
5,"(0, 1, 2, 3, 4)","[0.9175899480765564, 0.9203015496401156, 0.915...",0.917559,"(Avg. Area Income, Avg. Area House Age, Avg. A...",0.003532,0.002748,0.001374


<h2> Backward Feature Selection</h2>

In [66]:
#importing the necessary libraries
import mlxtend
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression
# Sequential Forward Selection(sfs)
sfs = SFS(LinearRegression(),
          k_features='best',
          forward=False,
          floating=False,
          verbose=2,
          scoring = 'r2',
          cv = 5)

In [67]:
sfs.fit(X, y)
print("Selected Features:",sfs.k_feature_names_)
print("Selected Features ID:",sfs.k_feature_idx_)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished

[2022-02-21 23:18:42] Features: 4/1 -- score: 0.9175829481912778[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished

[2022-02-21 23:18:42] Features: 3/1 -- score: 0.7970040383531473[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Selected Features: ('Avg. Area Income', 'Avg. Area House Age', 'Avg. Area Number of Rooms', 'Area Population')
Selected Features ID: (0, 1, 2, 4)


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished

[2022-02-21 23:18:42] Features: 2/1 -- score: 0.6124249771420225[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished

[2022-02-21 23:18:42] Features: 1/1 -- score: 0.4065836013812946

<h2>Step-wise Feature Selection</h2>

In [68]:
#importing the necessary libraries
import mlxtend
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression
# Sequential Forward Selection(sfs)
sfs = SFS(LinearRegression(),
          k_features='best',
          forward=True,
          floating=True,
          verbose=2,
          scoring = 'r2',
          cv = 5)

In [69]:
sfs.fit(X, y)
print("Selected Features:",sfs.k_feature_names_)
print("Selected Features ID:",sfs.k_feature_idx_)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished

[2022-02-21 23:18:42] Features: 1/5 -- score: 0.4065836013812946[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished

[2022-02-21 23:18:42] Features: 2/5 -- score: 0.6124249771420225[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs

Selected Features: ('Avg. Area Income', 'Avg. Area House Age', 'Avg. Area Number of Rooms', 'Area Population')
Selected Features ID: (0, 1, 2, 4)


In [70]:
df=pd.DataFrame.from_dict(sfs.get_metric_dict()).T
df

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
1,"(0,)","[0.4260936784765087, 0.4410126389901101, 0.364...",0.406584,"(Avg. Area Income,)",0.036804,0.028635,0.014317
2,"(0, 1)","[0.6264837396794845, 0.6566958614467223, 0.568...",0.612425,"(Avg. Area Income, Avg. Area House Age)",0.039491,0.030726,0.015363
3,"(0, 1, 4)","[0.799500115176274, 0.8146479457241405, 0.7881...",0.797004,"(Avg. Area Income, Avg. Area House Age, Area P...",0.012392,0.009641,0.004821
4,"(0, 1, 2, 4)","[0.917841711309842, 0.9202527128422192, 0.9153...",0.917583,"(Avg. Area Income, Avg. Area House Age, Avg. A...",0.003516,0.002736,0.001368
5,"(0, 1, 2, 3, 4)","[0.9175899480765564, 0.9203015496401156, 0.915...",0.917559,"(Avg. Area Income, Avg. Area House Age, Avg. A...",0.003532,0.002748,0.001374


<h2>Creating a new Dataframe using the features selected</h2> 
<p>Forward, backward and step-wise feature selections gave the same best features. So I have taken features selected by forward selection and created a new dataframe using them below</p>

In [71]:
features=list(sfs.k_feature_names_)
print(features)

['Avg. Area Income', 'Avg. Area House Age', 'Avg. Area Number of Rooms', 'Area Population']


In [72]:
new_data=dataset[features]
new_data['Price']=dataset['Price']
new_data

C:\Users\pyasw\AppData\Local\Temp\ipykernel_10068\810563098.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['Price']=dataset['Price']


,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Area Population,Price
0,79545.45857,5.682861,7.009188,23086.80050,1.059034e+06
1,79248.64245,6.002900,6.730821,40173.07217,1.505891e+06
2,61287.06718,5.865890,8.512727,36882.15940,1.058988e+06
3,63345.24005,7.188236,5.586729,34310.24283,1.260617e+06
4,59982.19723,5.040555,7.839388,26354.10947,6.309435e+05
...,...,...,...,...,...
4995,60567.94414,7.830362,6.137356,22837.36103,1.060194e+06
4996,78491.27543,6.999135,6.576763,25616.11549,1.482618e+06
4997,63390.68689,7.250591,4.805081,33266.14549,1.030730e+06
4998,68001.33124,5.534388,7.130144,42625.62016,1.198657e+06


<h2> Splitting the data into train and test </h2>

In [73]:
X_new=new_data.drop('Price',axis=1)
y_new=new_data['Price']

In [74]:
X_train,X_test,y_train,y_test=train_test_split(X_new,y_new,test_size=0.2)

<h2>Linear Regression</h2>

In [75]:
lr=LinearRegression()
lr.fit(X_train,y_train)
predict_lr=lr.predict(X_test)
x=r2_score(y_test,predict_lr)
print("R2",x)
adjusted_r2=calculate_adj_r2(x,X_test.count()[0],len(X))
print("Adj R2",adjusted_r2)
scores=cross_val_score(lr,X_train,y_train,scoring="r2",cv=crossvalidation,n_jobs=1)
print("Folds:"+str(len(scores))+",MSE:"+str(np.mean(np.abs(scores)))+",STD"+str(np.std(scores)))

R2 0.9167315121620713
Adj R2 1.0207911070607576
Folds:10,MSE:0.9174651433933038,STD0.00858288612039079


<h2> Ridge Regression </h2>

In [76]:
model_cv=RidgeCV(alphas=np.arange(.1,1,0.01),cv=5,scoring='r2')
model_cv.fit(X_train,y_train)
print("Best Alpha",model_cv.alpha_)
y_pred=model_cv.predict(X_test)
x=r2_score(y_test,y_pred)
print("R2",x)
adjusted_r2=calculate_adj_r2(x,X_test.count()[0],len(X_new))
print("Adj R2",adjusted_r2)

Best Alpha 0.8699999999999996
R2 0.9167307478025067
Adj R2 1.020791297911846


In [77]:
ridge=Ridge()
ridge.fit(X_train,y_train)
predict_ridge=ridge.predict(X_test)
x=r2_score(y_test, predict_ridge)
print("R2",x)
adjusted_r2 = calculate_adj_r2(x, X_test.count()[0],len(X))
print("Adj R2",adjusted_r2)

R2 0.9167306309919092
Adj R2 1.0207913270780011


<h2> Lasso Ridge </h2>

In [78]:
lasso=Lasso()
lasso.fit(X_train,y_train)
predict_lasso=lasso.predict(X_test)
x=r2_score(y_test, predict_lasso)
print("R2",x)
adjusted_r2 = calculate_adj_r2(x, X_test.count()[0],len(X))
print("Adj R2",adjusted_r2)

R2 0.916731496567186
Adj R2 1.0207911109546066


In [79]:
from sklearn.linear_model import LassoCV
model_l_cv = LassoCV(cv=5, random_state=0, max_iter=10000)
model_l_cv.fit(X_train, y_train)
lasso_best = Lasso(alpha=model_l_cv.alpha_)
lasso_best.fit(X_train, y_train)
Y_Pred=lasso_best.predict(X_test)

x=r2_score(y_test, Y_Pred)
print("R2",x)
adjusted_r2 = calculate_adj_r2(x, X_test.count()[0],len(X))
print("Adj R2",adjusted_r2)

R2 0.5977804470442624
Adj R2 1.1004292260441844


<h2>Quadratic Regression</h2>

In [80]:
quadratic=PolynomialFeatures(degree=2)
quadratic_features=quadratic.fit_transform(X_train)
quadratic.fit(quadratic_features,y_train)
quad_model=LinearRegression()
quad_model.fit(quadratic_features,y_train)
predict_quad=quad_model.predict(quadratic.fit_transform(X_test))
x=r2_score(y_test, predict_quad)
print("R2",x)
adjusted_r2 = calculate_adj_r2(x, X_test.count()[0],len(X))
print("Adj R2",adjusted_r2)

R2 0.9164787540340461
Adj R2 1.0208542176255906


<h2> Symbolic Regression </h2>

In [81]:
symbolic = SymbolicRegressor(population_size=5000,
                           generations=10, stopping_criteria=0.01,
                           p_crossover=0.7, p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05, p_point_mutation=0.1,
                           max_samples=0.9, verbose=1,
                           parsimony_coefficient=0.01, random_state=0)
symbolic.fit(X_train, y_train)
score_gp = symbolic.score(X_train, y_train)
print(score_gp)
Y_Pred=symbolic.predict(X_test)
x=r2_score(y_test, Y_Pred)
print("R2",x)
adjusted_r2 = calculate_adj_r2(x, X_test.count()[0],len(X))
print("Adj R2",adjusted_r2)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    31.03      7.86559e+45       15           188437           176523     34.98s
   1    21.45      6.92704e+15       59           152108           151627     28.80s
   2    26.98       1.2004e+14       43           145710           143525     23.95s
   3    28.49      9.18168e+13       19           142170           148516     23.42s
   4    30.67      2.11868e+20       41           132795           142975     18.87s
   5    42.17      3.08006e+16       31           129766           132949     18.46s
   6    37.83       1.8981e+15       23           123494           115465     11.27s
   7    37.63      1.53277e+18       27           122870           118395      7.73s
   8    41.58      1.05859e+19       27           122240           124060  

<h2> Symbolic Ridge Regression </h2>

In [82]:
function_set = ['add', 'sub', 'mul', 'div',
                'sqrt', 'log', 'abs', 'neg', 'inv',
                'max', 'min']
gp = SymbolicTransformer(generations=20, population_size=2000,
                         hall_of_fame=100, n_components=10,
                         function_set=function_set,
                         parsimony_coefficient=0.0005,
                         max_samples=0.9, verbose=1,
                         random_state=0, n_jobs=3)
gp.fit(X_new[:6000],y_new[:6000])

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    13.74         0.360312       34         0.902974         0.909516     28.90s
   1    13.87         0.594559       38         0.919293         0.915683     16.36s
   2    23.65         0.695632       41          0.93924         0.935205     17.37s
   3    31.92         0.781227       41         0.939725         0.930945     17.37s
   4    32.37          0.81823       27         0.947213         0.949189     16.63s
   5    29.03         0.813787       35         0.950309         0.951842     14.34s
   6    25.75         0.817743       33         0.951224         0.942149     13.37s
   7    24.17         0.831942       45         0.952459         0.948024     12.25s
   8    22.92         0.832397       42         0.953166         0.949809  

SymbolicTransformer(function_set=['add', 'sub', 'mul', 'div', 'sqrt', 'log',
                                  'abs', 'neg', 'inv', 'max', 'min'],
                    max_samples=0.9, n_jobs=3, parsimony_coefficient=0.0005,
                    population_size=2000, random_state=0, verbose=1)

In [83]:

gp_features = gp.transform(X_new)
new_data = np.hstack((X_new, gp_features))

In [84]:
est = Ridge()
est.fit(new_data[:3500],y_new[:3500])
print(est.score(new_data[3500:],y_new[3500:]))
Y_Pred= est.predict(new_data[3500:])

x=r2_score(y_new[3500:], Y_Pred)
print("R2",x)
adjusted_r2 = calculate_adj_r2(x, len(new_data[3500:]),len(X))
print("Adj R2",adjusted_r2)
print(max_error(y_new[3500:], Y_Pred))

0.917592566073667
R2 0.917592566073667
Adj R2 1.0352838456028486
364050.3040387565


<h2> Symbolic Lasso Ridge </h2>

In [85]:
est = Lasso()
est.fit(new_data[:3500],y_new[:3500])
print(est.score(new_data[3500:],y_new[3500:]))
Y_Pred= est.predict(new_data[3500:])

x=r2_score(y_new[3500:], Y_Pred)
print("R2",x)
adjusted_r2 = calculate_adj_r2(x, len(new_data[3500:]),len(X))
print("Adj R2",adjusted_r2)
print(max_error(y_new[3500:], Y_Pred))

0.917599121082713
R2 0.917599121082713
Adj R2 1.0352810389880072
365772.8035074114


C:\Users\pyasw\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.646e+13, tolerance: 4.373e+10
  model = cd_fast.enet_coordinate_descent(
